1. Using different optimizer strategy - 
    a. Stochastic Weight Averaging, 
    b. Reinitializing Transformer Layers (Pooler and a few last layers) 
    c. Pooler block representation on the Intermidiate layers (

2. Train A bigger Deberta Model - Optimization technique (quantization etc)
3. Train A roberta model - reason cus it is smaller and much faster!
4. train a distilbert and use a fast tokenizer - for efficiency track 

Difficult
5. Knowledge Distillation (Multiple teacher models) - requires pseodo label, KD Loss Function... 
6. 


Plan
1. Do SWA (con do on composer but this is a different library - might not be consistent)
2. Train Deberta-Large (using optimization method) - finetuning with the best combination techniques we have found so far for deberta.
3. Do knowledge Distilation from Deberta-Base & Deberta Large to Deberta-XSmall/Deberta-Small
4. Hyperparameter tuning on Deberta-XSmall/Deberta-Small on a smaller sample size. 

In [1]:
!pip install iterative-stratification
!pip install coolname

In [2]:
import os
import gc
import copy
import time
import random
import string
import joblib

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils|
from tqdm import tqdm
from collections import defaultdict
from feedback_custom_funtions import loss_fn, optimizer_setup, FeedBackDataset, RMSELoss, compute_metrics
from model_building import MeanPooling, MaxPooling, MinPooling, AttentionPooling, FeedBackModel
from coolname import generate_slug

# For splitting data
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import DataCollatorWithPadding
from transformers import Trainer, TrainingArguments
from transformers.modeling_outputs import SequenceClassifierOutput

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

## Training Confg

In [3]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

hash_name = generate_slug(3)

config = {"seed": 42,
          "epochs": 5,
          "debug" : False,
          "model_name": "roberta-base",
          "PoolingLayer": AttentionPooling(768),
          "group" : "roberta-base-AP-LLRD" ,
          "loss_type": "smooth_l1", # ['mse', 'rmse', 'smooth_l1']
          "train_batch_size": 8,
          "valid_batch_size": 16,
          "fp16_enable": False,
          "max_length": 512,
          "layerwise" : True,
          "learning_rate": 1e-5,
          "decoder_lr": 1e-4,
          "weight_decay": 1e-6,
          "n_fold": 4,
          "n_accumulate": 1,
          "max_grad_norm": 1000,
          "num_classes": 6,
          "target_cols": ["cohesion", "syntax", "vocabulary", 
                          "phraseology", "grammar", "conventions"],
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
          "hash_name": hash_name,
          "competition": "FeedBack3",
          "_wandb_kernel": "hazrul"
          }

set_seed(config['seed'])

In [4]:
if not config["debug"]:    
    import wandb

    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        api_key = user_secrets.get_secret("WANDB_API_KEY")
        wandb.login(key=api_key)
        anony = None
        print("wandb Logged in Successfully")
    except:
        anony = "must"
        print('If you want to use your W&B account, go to Add-ons -> Secrets and provide your W&B access token. Use the Label name as wandb_api. \nGet your W&B access token from here: https://wandb.ai/authorize')
else:
    os.environ["WANDB_DISABLED"] = "true"
    print("Debugging...")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb Logged in Successfully


## Data Loading

In [5]:
df = pd.read_csv("../input/feedback-prize-english-language-learning/train.csv")
mskf = MultilabelStratifiedKFold(n_splits=config['n_fold'], shuffle=True, random_state=config['seed'])

for fold, (train_idx, val_idx) in enumerate(mskf.split(X=df, y=df[config['target_cols']])):
    df.loc[val_idx , "kfold"] = int(fold)
    
df["kfold"] = df["kfold"].astype(int)
df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,kfold
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,2
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,0
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,1
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0,3
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5,3


In [6]:
tokenizer = AutoTokenizer.from_pretrained(config["model_name"])
config["tokenizer"] = tokenizer

collate_fn = DataCollatorWithPadding(tokenizer=config['tokenizer'])

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

## Training Setup

In [7]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(inputs['input_ids'], inputs['attention_mask'])
        loss = loss_fn(outputs.logits, inputs['target'], loss_type=config['loss_type'])
        return (loss, outputs) if return_outputs else loss

In [8]:
for fold in range(0, config['n_fold']):
    print(f"========== Fold: {fold} ==========")
    
    if not config["debug"]:
        run = wandb.init(project=config['competition'], 
                         config=config,
                         job_type='Train',
                         group=config['group'],
                         tags=[config['model_name'], config['loss_type']],
                         name=f'{config["hash_name"]}-fold-{fold}',
                         anonymous='must')

    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    train_dataset = FeedBackDataset(df_train, tokenizer=config['tokenizer'], max_length=config['max_length'], target_label = config["target_cols"])
    valid_dataset = FeedBackDataset(df_valid, tokenizer=config['tokenizer'], max_length=config['max_length'], target_label = config["target_cols"])

    model = FeedBackModel(config['model_name'], config["num_classes"], PoolingLayer = config["PoolingLayer"]).to(config['device'])

    # Define Optimizer and Scheduler
    optimizer, scheduler = optimizer_setup(model=model, 
                                           config=config, 
                                           train_dataset_size =len(train_dataset),
                                           layerwise = config["layerwise"]
                                          )

    training_args = TrainingArguments(
        output_dir=f"outputs-{fold}/",
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        per_device_train_batch_size=config['train_batch_size'],
        per_device_eval_batch_size=config['valid_batch_size'],
        num_train_epochs= config['epochs'],
        learning_rate= config['learning_rate'],
        weight_decay= config['weight_decay'],
        gradient_accumulation_steps=config['n_accumulate'],
        max_grad_norm=config['max_grad_norm'],
        seed= config['seed'],
        fp16  = config["fp16_enable"],
        fp16_full_eval  = config["fp16_enable"],
        half_precision_backend = "cuda_amp",
        group_by_length = True,
        metric_for_best_model= 'eval_mcrmse',
        load_best_model_at_end=True,
        greater_is_better=False,
        save_strategy="epoch",
        save_total_limit=1,
        report_to = "wandb",
        label_names = ["target"]
    )


    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=collate_fn,
        optimizers=(optimizer, scheduler),
        compute_metrics=compute_metrics
    )

    trainer.train()

    #evaluation = trainer.evaluate()
    #run.log({"score_mcrmse": evaluation["eval_mcrmse"], "eval_runtime": evaluation["eval_runtime"]})
    
    if not config["debug"]:
        run.finish()

    del model, train_dataset, valid_dataset

    torch.cuda.empty_cache()
    gc.collect()

wandb: Currently logged in as: hazrulakmal. Use `wandb login --relogin` to force relogin


========== Fold: 0 ==========


Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
***** Running training *****
  Num examples = 2933
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1835
Automatic Weights & Biases logging enabled, to d

Epoch,Training Loss,Validation Loss,Mcrmse,Cohesion Rmse,Syntax Rmse,Vocabulary Rmse,Phraseology Rmse,Grammar Rmse,Conventions Rmse
1,0.539000,0.193958,0.630112,0.627050,0.599265,0.550955,0.655828,0.688224,0.659351
2,0.131200,0.107456,0.464140,0.495524,0.451732,0.417282,0.470941,0.493352,0.456009
3,0.115900,0.113683,0.477408,0.510497,0.463868,0.423190,0.496818,0.517918,0.452157
4,0.108900,0.111055,0.472168,0.501292,0.459783,0.423320,0.473591,0.505691,0.469330
5,0.101300,0.104489,0.457654,0.492714,0.449945,0.413291,0.464335,0.479210,0.446429


...

========== Fold: 3 ==========
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://huggingface.co/roberta-base/r

Epoch,Training Loss,Validation Loss,Mcrmse,Cohesion Rmse,Syntax Rmse,Vocabulary Rmse,Phraseology Rmse,Grammar Rmse,Conventions Rmse
1,0.511200,0.129203,0.510959,0.558011,0.488387,0.507209,0.479431,0.550225,0.482488
2,0.128300,0.129257,0.511139,0.549146,0.540371,0.483972,0.487570,0.507198,0.498576
3,0.111100,0.112411,0.475573,0.508389,0.463075,0.430437,0.459601,0.514446,0.477490
4,0.101300,0.108855,0.467917,0.495785,0.465173,0.430227,0.459563,0.500413,0.456339
5,0.091900,0.109735,0.469843,0.498295,0.467552,0.431566,0.461638,0.502050,0.457959


***** Running Evaluation *****
  Num examples = 978
  Batch size = 16
Saving model checkpoint to outputs-3/checkpoint-367
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 978
  Batch size = 16
Saving model checkpoint to outputs-3/checkpoint-734
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 978
  Batch size = 16
Saving model checkpoint to outputs-3/checkpoint-1101
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [outputs-3/checkpoint-367] due to args.save_total_limit
Deleting older checkpoint [outputs-3/checkpoint-734] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 978
  Batch size = 16
Saving model checkpoint to outputs-3/checkpoint-1468
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Deleting older checkpoint [outputs-3/checkpoint-1101] due to args.

eval/cohesion_rmse,█▇▂▁▁
eval/conventions_rmse,▅█▅▁▁
eval/grammar_rmse,█▂▃▁▁
eval/loss,██▂▁▁
eval/mcrmse,██▂▁▁
eval/phraseology_rmse,▆█▁▁▂
eval/runtime,▁▅▅█▇
eval/samples_per_second,█▄▄▁▂
eval/steps_per_second,█▃▃▁▂
eval/syntax_rmse,▃█▁▁▁
eval/vocabulary_rmse,█▆▁▁▁
